### Обработка

In [1]:
from corpus import ConllCorpusReaderX

In [2]:
import os

def prepare(dataset):
    factrueval_dev_tokens = dict()
    factrueval_dev_tokens_list = []
    factrueval_dev_spans = dict()
    factrueval_dev_objects = dict()
    for file in os.listdir('./factrueval2016_dataset/'+ dataset + '/'):
        if file.endswith('tokens'):
            with open('./factrueval2016_dataset/' + dataset + '/' + file, 'r+', encoding='utf-8') as file_obj:
                lines = file_obj.readlines()
                tokens = [line.rstrip().split() for line in lines if line.rstrip().split() != []]
                for token in tokens:
                    factrueval_dev_tokens[int(token[0])] = token[1:]
                tokens = [line.rstrip().split() for line in lines]
                for token in tokens:
                    factrueval_dev_tokens_list.append(token)

        if file.endswith('spans'):
            with open('./factrueval2016_dataset/' + dataset + '/' + file, 'r+', encoding='utf-8') as file_obj:
                spans = [line.rstrip().split() for line in file_obj.readlines() if line.rstrip().split() != []]
                for span in spans:
                    factrueval_dev_spans[span[0]] = span[1:]

        if file.endswith('objects'):
            with open('./factrueval2016_dataset/' + dataset + '/' + file, 'r+', encoding='utf-8') as file_obj:
                objects = [line.rstrip().split('#')[0].split() for line in file_obj.readlines() if line.rstrip().split() != []]
                for obj in objects:
                    factrueval_dev_objects[obj[0]] = obj[1:]

    all_ne = []
    for key, value in factrueval_dev_objects.items():
        spans = value[1:]
        ne = value[0]
        all_tokens = []
        for span in spans:
            span_obj = factrueval_dev_spans[span]
            token = int(span_obj[3])
            num_of_tokens = int(span_obj[4])
            for i in range(num_of_tokens):
                all_tokens.append(token + i)
        all_ne.append([ne, sorted(all_tokens)])

    for ne_tokens in all_ne:
        ne = ne_tokens[0]
        token = ne_tokens[1]
        for i in range(len(token)):
            if token[i] in factrueval_dev_tokens.keys():
                if len(token) == 1:
                    factrueval_dev_tokens[token[i]].append("S-" + ne)
                elif (i == 0 and token[i + 1] - token[i] > 1) or (i == len(token) - 1 and token[i] - token[i - 1] > 1) or (token[i] - token[i - 1] > 1 and token[i + 1] - token[i] > 1):
                    factrueval_dev_tokens[token[i]].append("S-" + ne)
                elif (i == 0  and token[i + 1] - token[i] == 1) or (i != len(token) - 1 and token[i] - token[i - 1] > 1 and token[i + 1] - token[i] == 1):
                    factrueval_dev_tokens[token[i]].append("B-" + ne)
                elif (i == len(token) - 1 and token[i] - token[i - 1] == 1) or (i != 0 and token[i] - token[i - 1] == 1 and token[i + 1] - token[i] > 1):
                    factrueval_dev_tokens[token[i]].append("E-" + ne)
                else: 
                    factrueval_dev_tokens[token[i]].append("I-" + ne)

    for i in range(len(factrueval_dev_tokens_list)):
        if factrueval_dev_tokens_list[i] == []:
            continue
        number_of_token = factrueval_dev_tokens_list[i][0]
        if int(number_of_token) in factrueval_dev_tokens.keys() and len(factrueval_dev_tokens[int(number_of_token)]) >= 4:
            ne = factrueval_dev_tokens[int(number_of_token)][3]
            factrueval_dev_tokens_list[i].append(ne)
        else:
            factrueval_dev_tokens_list[i].append("O")

    final = []
    for el in factrueval_dev_tokens_list:
        if el == []:
            final.append(el)
        else:
            final.append([el[3], el[1], el[2], el[4]])
    return final

In [3]:
devset = prepare('devset')
with open('./factrueval2016_dataset/devset.txt', 'w+', encoding='utf-8') as file:
    file.write("-DOCSTART- O\n")
    for line in devset:
        if line == []:
            file.write("\n")
        else:
            file.write("{} {} {} {}\n".format(*line))    

In [4]:
testset = prepare('testset')
with open('./factrueval2016_dataset/testset.txt', 'w+', encoding='utf-8') as file:
    file.write("-DOCSTART- O\n")
    for line in testset:
        if line == []:
            file.write("\n")
        else:
            file.write("{} {} {} {}\n".format(*line))    

In [5]:
factrueval_devset = ConllCorpusReaderX('./factrueval2016_dataset/', 
                              fileids='devset.txt', 
                              columntypes=['words', 'offset', 'len', 'ne'])

factrueval_testset = ConllCorpusReaderX('./factrueval2016_dataset/', 
                              fileids='testset.txt', 
                              columntypes=['words', 'offset', 'len', 'ne'])

### Демонстрация работы

In [6]:
print(factrueval_devset.get_tags(tags=['words', 'ne', 'offset', 'len'])[:7])

[('В', 'O', '0', '1'), ('понедельник', 'O', '2', '11'), ('28', 'O', '14', '2'), ('июня', 'O', '17', '4'), ('у', 'O', '22', '1'), ('здания', 'O', '24', '6'), ('мэрии', 'B-Org', '31', '5')]


In [7]:
print(factrueval_devset.words())
print(len(factrueval_devset.words()))

['В', 'понедельник', '28', 'июня', 'у', 'здания', ...]
30940


In [8]:
print(factrueval_devset.get_ne())

[('В', 'O'), ('понедельник', 'O'), ('28', 'O'), ...]


In [13]:
import numpy as np
from generator import Generator

words = [[el] for el in factrueval_devset.words()[:4000]]
gen = Generator(column_types=['word'], context_len=2)
X = gen.generate(words, path='./factrueval_devset.npy')
y = np.array([el[1] for el in factrueval_devset.get_ne()[:4000]])

In [14]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

In [15]:
clf.fit(X[:3000], y[:3000])
y_pred = clf.predict(X[3000:4000])

In [16]:
right = 0
total_a = 0
total_b = 0
for a, b in zip(y_pred, y[3000:4000]):
    if a != 'O':
        total_a += 1
    if b != 'O':
        total_b += 1    
    if a != 'O' and a == b:
        right += 1
accuracy = right/total_a
recall = right/total_b
f = 2 * accuracy * recall /(accuracy + recall)
print(f)

0.5340314136125655
